In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import numpy as np
import copy

# Settings for printing numpy arrays
np.set_printoptions(precision=4, edgeitems=50, suppress=True)
np.core.arrayprint._line_width = 250

# Simplex Method

Below is defined a function which comprises the Simplex Method to solve Linear Programming Problems where we have to minimize or maximize an objective function under some restrictions.

In [3]:
def simplex(A, b, c, variables, itera = 1):
    
    # Printing our initial parameters for this iteration
    print("\n\n\033[1mIteration ", itera, "\033[0m")
    print("[\033[94m x_B \033[0m, \033[91m x_N \033[0m] = [ \033[94m"
          + ', '.join(variables[:len(b)]) + "\033[0m, \033[91m" + ', '.join(variables[len(b):]) + "\033[0m ]\n")
    print("Matrix A")
    print(A)
    print("\nVector b")
    print(b)
    print("\nVector c")
    print(c)
    
    # Partioning our matrix A into a basic matrix B and a not basic matrix N
    m, n = np.shape(A)
    B = A[:m, :m]
    N = A[:m, m:]
    
    # Calculating and printing our Basis, Basis Inverse and Basic Feasible Solution BFS
    B_inv = np.linalg.inv(B)
    x_b = np.array(B_inv@b)
    x_b = x_b[0, :]
    x_n = np.zeros(n - m)
    x = np.concatenate((x_b, x_n), axis = 0)
    print("\nBasis")
    print(B)
    print("\nBasis Inverse")
    print(B_inv)
    print("\nBFS")
    print(x)
    
    # Calculating the z's minus c's and finding the maximum one with its index k
    j_index = [i for i in range(m, n)]
    c_b = c[:m]
    zs = np.zeros(len(j_index))
    z_minus_c = np.zeros(len(j_index))
    
    for i in range(len(j_index)):
        a_j = A[:,j_index[i]]
        zs[i] = c_b@B_inv@a_j
        z_minus_c[i] = zs[i] - c[j_index[i]]
        
    z_minus_c_max = max(z_minus_c)
    k = m + np.argmax(z_minus_c)
    
    # Validating if we stop or not the optimization
    if z_minus_c_max <= 0:   # We stop
        
        # Printing the final results
        print("\n\n\033[1mOptimality reached\033[0m")
        print("\nThe optimal BFS is")
        results_ls = [str(round(x_i, 4)) for x_i in x]
        print("\033[92m[ " + ', '.join(variables) + " ] = [ " + ', '.join(results_ls) + " ]\033[0m")
        perf_z = c.T@x
        print("\nWith performance z =", perf_z, "\n\n")
        
        indexes_variables = [int(var[1:]) for var in variables]
        dict_solution = dict(zip(indexes_variables, [x_i for x_i in x]))
        sorted_dict_solution = dict(sorted(dict_solution.items(), key = lambda x:x[0]))
        solution = np.array(list(sorted_dict_solution.values()))
        #print(solution)
        return solution
        
    else:   # We continue
        
        # Calculating and printing the y_ki's
        y_k = np.ravel(B_inv@A[:,k])
        print("\nk =", k + 1, "-> column no.", k + 1, "of Matrix A (", variables[k], ")")
        print("y_k")
        print(y_k)
        
        # Analyzing if the optimal BFS is or not boundable by the condition y_k > 0
        flag = True
        aux_counter = 0
        for y_i in y_k:
            if y_i > 0:
                aux_counter = aux_counter + 1      
        if aux_counter == 0:
            flag = False

        if flag:   # Boundable
            
            # Calculating and printing x_k by the minimum quotient (current BFS divided by the y_k > 0) with its index r
            quot_ls = list()
            index_r_ls = list()
            for i in range(m):
                if y_k[i] > 0:
                    quot_ls.append(x[i] / y_k[i])
                    index_r_ls.append(i)
            quot_arr = np.array(quot_ls)
            index_r_arr = np.array(index_r_ls)
            r = index_r_arr[np.argmin(quot_arr)]
            x_Br = min(quot_arr)
            print("\nr =", r + 1, "-> column no.", r + 1, "of Basis (", variables[r], ")")
            print("x_Br")
            print(x_Br)

            # Exchanging the basic variable with index r and the not basic variable with index k, of the matrix A partitions
            var_aux = np.array(B[:, r])
            B[:, r] = N[:, k - m]
            N[:, k - m] = var_aux
            A = np.concatenate((B, N), axis = 1)
            
            # Exchanging the values located on indexes r and k of the vector c
            var_aux_c = c[r]
            c[r] = c[k]
            c[k] = var_aux_c
            
            # Exchanging the indexes
            print("\n\033[94m", variables[k], "enters\033[0m and \033[91m", variables[r], "leaves\033[0m the basis")
            aux_index = variables[r]
            variables[r] = variables[k]
            variables[k] = aux_index
            
            # Recursive call
            return simplex(A, b, c, variables, itera + 1)

        else:   # Not boundable
            
            print("\n\n\033[1mOptimization process stopped :(\033[0m")
            print("\nThe optimal BFS is not boundable\n\n")
            
            return False
            

Once we have already defined the previous function, we are able to apply it.

#### Example 1:
Solve the next linear programming problem:

$$min \hspace{0.5cm} -x_1 -3x_2$$
$$under \hspace{0.5cm} 2x_1 + 3x_2 \leq 6$$
$$\hspace{1cm}-x_1 + x_2 \leq 1$$
$$\hspace{1cm}x_1, x_2 \geq 0$$

First, we introduce our lack variables $x_3, x_4$ and the problem can be rewritten as
$$min \hspace{0.5cm} -x_1 -3x_2 + 0x_3 + 0x_4$$
$$under \hspace{0.5cm} 2x_1 + 3x_2 + x_3 + 0x_4 = 6$$
$$\hspace{1cm}-x_1 + x_2 + 0x_3 + x_4 = 1$$
$$\hspace{1cm}x_1, x_2, x_3, x_4 \geq 0$$

Then, initially considering $x_3, x_4$ as the basic variables and $x_1, x_2$ as the non basic ones, we set our parameters that will enter to the method as presented below:
\begin{equation} A = [B | N] = 
\left[
\begin{array}{rr|rr}
1 & 0 & 2 & 3\\
0 & 1 & -1 & 1\\
\end{array}
\right]
\end{equation}

\begin{equation} b = 
\left[
\begin{array}{r}
6\\
1\\
\end{array}
\right]
\end{equation}


\begin{equation} c = [c_{B} | c_{N}] =
\left[
\begin{array}{rr|rr}
0 & 0 & -1 & -3\\
\end{array}
\right]
\end{equation}

In [4]:
A = np.matrix([[1, 0, 2, 3], [0, 1, -1, 1]])
b = np.array([6, 1])
c = np.array([0, 0, -1, -3])
variables = ["x3", "x4", "x1", "x2"]

In [5]:
simplex(copy.deepcopy(A), copy.deepcopy(b), copy.deepcopy(c), copy.deepcopy(variables))



Iteration  1 
[ x_B ,  x_N ] = [ x3, x4, x1, x2 ]

Matrix A
[[ 1  0  2  3]
 [ 0  1 -1  1]]

Vector b
[6 1]

Vector c
[ 0  0 -1 -3]

Basis
[[1 0]
 [0 1]]

Basis Inverse
[[1. 0.]
 [0. 1.]]

BFS
[6. 1. 0. 0.]

k = 4 -> column no. 4 of Matrix A ( x2 )
y_k
[3. 1.]

r = 2 -> column no. 2 of Basis ( x4 )
x_Br
1.0

 x2 enters and  x4 leaves the basis


Iteration  2 
[ x_B ,  x_N ] = [ x3, x2, x1, x4 ]

Matrix A
[[ 1  3  2  0]
 [ 0  1 -1  1]]

Vector b
[6 1]

Vector c
[ 0 -3 -1  0]

Basis
[[1 3]
 [0 1]]

Basis Inverse
[[ 1. -3.]
 [ 0.  1.]]

BFS
[3. 1. 0. 0.]

k = 3 -> column no. 3 of Matrix A ( x1 )
y_k
[ 5. -1.]

r = 1 -> column no. 1 of Basis ( x3 )
x_Br
0.6

 x1 enters and  x3 leaves the basis


Iteration  3 
[ x_B ,  x_N ] = [ x1, x2, x3, x4 ]

Matrix A
[[ 2  3  1  0]
 [-1  1  0  1]]

Vector b
[6 1]

Vector c
[-1 -3  0  0]

Basis
[[ 2  3]
 [-1  1]]

Basis Inverse
[[ 0.2 -0.6]
 [ 0.2  0.4]]

BFS
[0.6 1.6 0.  0. ]


Optimality reached

The optimal BFS is
[ x1, x2, x3, x4 ] = [ 0.6, 1.6, 0.

array([0.6, 1.6, 0. , 0. ])

#### Example 2:
Solve the next linear programming problem:

$$max \hspace{0.5cm} 5x_1 + 4x_2$$
$$under \hspace{0.5cm} x_1 + 2x_2 \leq 6$$
$$\hspace{1cm}2x_1 - x_2 \leq 4$$
$$\hspace{1cm}5x_1 + 3x_2 \leq 15$$
$$\hspace{1cm}x_1, x_2 \geq 0$$

First, we introduce our lack variables $x_3, x_4, x_5$ and then multiply by $-1$ the objective function in order to turn the maximization problem into a minimization one. So the problem can be rewritten as
$$min \hspace{0.5cm} -5x_1 -4x_2 + 0x_3 + 0x_4 + 0x_5$$
$$under \hspace{0.5cm} x_1 + 2x_2 + x_3 + 0x_4 + 0x_5 = 6$$
$$\hspace{1cm}2x_1 - x_2 + 0x_3 + x_4 + 0x_5 = 4$$
$$\hspace{1cm}5x_1 + 3x_2 + 0x_3 + 0x_4 +x_5 = 15$$
$$\hspace{1cm}x_1, x_2, x_3, x_4, x_5 \geq 0$$

Then, initially considering $x_3, x_4, x_5$ as the basic variables and $x_1, x_2$ as the non basic ones, we set our parameters that will enter to the method as presented below:
\begin{equation} A = [B | N] = 
\left[
\begin{array}{rrr|rr}
1 & 0 & 0 & 1 & 2\\
0 & 1 & 0 & 2 & -1\\
0 & 0 & 1 & 5 & 3\\
\end{array}
\right]
\end{equation}

\begin{equation} b = 
\left[
\begin{array}{r}
6\\
4\\
15\\
\end{array}
\right]
\end{equation}


\begin{equation} c = [c_{B} | c_{N}] =
\left[
\begin{array}{rrr|rr}
0 & 0 & 0 & -5 & -4\\
\end{array}
\right]
\end{equation}

In [6]:
A = np.matrix([[1, 0, 0, 1, 2], [0, 1, 0, 2, -1], [0, 0, 1, 5, 3]])
b = np.array([6, 4, 15])
c = np.array([0, 0, 0, -5, -4])
variables = ["x3", "x4", "x5", "x1", "x2"]

In [7]:
simplex(copy.deepcopy(A), copy.deepcopy(b), copy.deepcopy(c), copy.deepcopy(variables))



Iteration  1 
[ x_B ,  x_N ] = [ x3, x4, x5, x1, x2 ]

Matrix A
[[ 1  0  0  1  2]
 [ 0  1  0  2 -1]
 [ 0  0  1  5  3]]

Vector b
[ 6  4 15]

Vector c
[ 0  0  0 -5 -4]

Basis
[[1 0 0]
 [0 1 0]
 [0 0 1]]

Basis Inverse
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

BFS
[ 6.  4. 15.  0.  0.]

k = 4 -> column no. 4 of Matrix A ( x1 )
y_k
[1. 2. 5.]

r = 2 -> column no. 2 of Basis ( x4 )
x_Br
2.0

 x1 enters and  x4 leaves the basis


Iteration  2 
[ x_B ,  x_N ] = [ x3, x1, x5, x4, x2 ]

Matrix A
[[ 1  1  0  0  2]
 [ 0  2  0  1 -1]
 [ 0  5  1  0  3]]

Vector b
[ 6  4 15]

Vector c
[ 0 -5  0  0 -4]

Basis
[[1 1 0]
 [0 2 0]
 [0 5 1]]

Basis Inverse
[[ 1.  -0.5  0. ]
 [ 0.   0.5  0. ]
 [-0.  -2.5  1. ]]

BFS
[4. 2. 5. 0. 0.]

k = 5 -> column no. 5 of Matrix A ( x2 )
y_k
[ 2.5 -0.5  5.5]

r = 3 -> column no. 3 of Basis ( x5 )
x_Br
0.9090909090909091

 x2 enters and  x5 leaves the basis


Iteration  3 
[ x_B ,  x_N ] = [ x3, x1, x2, x4, x5 ]

Matrix A
[[ 1  1  2  0  0]
 [ 0  2 -1  1  0]
 [ 0  5  3  0 

array([1.7143, 2.1429, 0.    , 2.7143, 0.    ])

#### Example 3:
Solve the next linear programming problem:

$$max \hspace{0.5cm} 3x_1 + 2x_2$$
$$under \hspace{0.5cm} 2x_1 - 3x_2 \leq 3$$
$$\hspace{1cm}-x_1 + x_2 \leq 5$$
$$\hspace{1cm}x_1, x_2 \geq 0$$

First, we introduce our lack variables $x_3, x_4$ and then multiply by $-1$ the objective function in order to turn the maximization problem into a minimization one. So the problem can be rewritten as
$$min \hspace{0.5cm} -3x_1 -2x_2 + 0x_3 + 0x_4$$
$$under \hspace{0.5cm} 2x_1 - 3x_2 + x_3 + 0x_4 = 3$$
$$\hspace{1cm}-x_1 + x_2 + 0x_3 + x_4 = 5$$
$$\hspace{1cm}x_1, x_2, x_3, x_4 \geq 0$$

Then, initially considering $x_3, x_4$ as the basic variables and $x_1, x_2$ as the non basic ones, we set our parameters that will enter to the method as presented below:
\begin{equation} A = [B | N] = 
\left[
\begin{array}{rr|rr}
1 & 0 & 2 & -3\\
0 & 1 & -1 & 1\\
\end{array}
\right]
\end{equation}

\begin{equation} b = 
\left[
\begin{array}{r}
3\\
5\\
\end{array}
\right]
\end{equation}


\begin{equation} c = [c_{B} | c_{N}] =
\left[
\begin{array}{rr|rr}
0 & 0 & -3 & -2\\
\end{array}
\right]
\end{equation}

In [8]:
A = np.matrix([[1, 0, 2, -3], [0, 1, -1, 1]])
b = np.array([3, 5])
c = np.array([0, 0, -3, -2])
variables = ["x3", "x4", "x1", "x2"]

In [9]:
simplex(copy.deepcopy(A), copy.deepcopy(b), copy.deepcopy(c), copy.deepcopy(variables))



Iteration  1 
[ x_B ,  x_N ] = [ x3, x4, x1, x2 ]

Matrix A
[[ 1  0  2 -3]
 [ 0  1 -1  1]]

Vector b
[3 5]

Vector c
[ 0  0 -3 -2]

Basis
[[1 0]
 [0 1]]

Basis Inverse
[[1. 0.]
 [0. 1.]]

BFS
[3. 5. 0. 0.]

k = 3 -> column no. 3 of Matrix A ( x1 )
y_k
[ 2. -1.]

r = 1 -> column no. 1 of Basis ( x3 )
x_Br
1.5

 x1 enters and  x3 leaves the basis


Iteration  2 
[ x_B ,  x_N ] = [ x1, x4, x3, x2 ]

Matrix A
[[ 2  0  1 -3]
 [-1  1  0  1]]

Vector b
[3 5]

Vector c
[-3  0  0 -2]

Basis
[[ 2  0]
 [-1  1]]

Basis Inverse
[[0.5 0. ]
 [0.5 1. ]]

BFS
[1.5 6.5 0.  0. ]

k = 4 -> column no. 4 of Matrix A ( x2 )
y_k
[-1.5 -0.5]


Optimization process stopped :(

The optimal BFS is not boundable




False